---
# Using Flink to Read, Process, and Write Streams

![overview](safety_and_security_with_flink.png)

- Flink is fully integrated into Streaming Data Platform and it can be used to read, process, and write streams, including video streams.

---
# Multi Video Grid Flink Job

![overview](multi_video_grid_overview.png)

- Live video feeds from 4 cameras are continuously ingested into Streaming Data Platform (Pravega).
  This original video is ingested into the stream named *safety-and-security-video*.

- The Flink Multi Video Grid job is a streaming job that scales down the original videos, and combines them into a 2x2 grid, and produces a new video stream named *multi-video-grid-output*.

- All operations are parallelized across multiple servers. 
  By increasing the number of Flink containers (Task Managers), the number of cameras, frame rate, and resolution can be increased.
  This demo uses a parallelism of 16 and uses 16 CPU cores.

- Watermarks are used to correctly align videos by time window.

- Pravega and Flink ordering guarantees are utilized to ensure that output images are ordered correctly.

- This demo produces output images of 1280x720 resolution, JPEG format, at 4 frames per second.
  It achieves an end-to-end latency of 1.2 to 1.5 seconds.  

---

### How to use this notebook
1. [Install dependencies](install_dependencies.ipynb).
2. [safety_and_security_download.ipynb](safety_and_security_download.ipynb).
3. [safety_and_security_transcode_from_video.ipynb](safety_and_security_transcode_from_video.ipynb).
4. [Begin ingesting data](safety_and_security_ingest.ipynb).
5. Run Flink job multi-video-grid-job.
6. [Run continuous index builder job](multi_video_grid_index_builder.ipynb).
7. Run video player. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Initialize

In [3]:
#### Import dependencies
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import VideoPlayer
import os
imp.reload(video);

### Define Pravega stream parameters

gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'multi-video-grid-output'

### Initialize connection to Pravega GRPC Gateway

pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
### Video Player

In [5]:
VideoPlayer(pravega_client=pravega_client, scope=scope, stream=stream).interact()

load_index: Reading index stream multi-video-grid-output-index
